In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from requests import get
from random import randint
PATH = ("C:/Users/jw156/Ironhack/CareerHack/chromedriver.exe")
import pandas as pd
from datetime import date, timedelta
import time

1. Scrape the headline, first para.
2. date: newspaper: tabload or broad: text: sentiment
3. Shuffle the dataset and kmeans.
shuffle the dataset, before and after

### Finding the dates between Date Range 

Claim: The lockdown began on 16 March 2020, when Matt Hancock told the House of Commons that all unnecessary social contact should cease.

Look at articles 6 months before and 6 months afterwards.

In [2]:
# Start 6 months before: '16 September 2018' 
# Finish 6 months afterwards: '16 September 2019'

In [3]:
sdate = date(2019, 9, 16)   # start date
edate = date(2020, 9, 16)   # end date

In [4]:
# Frequency weekly, get all the weekly articles

In [5]:
all_search_dates = pd.date_range(sdate,edate-timedelta(days=1),freq='w')

In [6]:
# Changing format so it matches google requirements

In [7]:
search_days = [day.strftime('%m-%d-%Y') for day in all_search_dates]

In [8]:
len(search_days)

52

# Selenium Search Functions

In [9]:
def search_selenium(subject, website):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    driver = webdriver.Chrome(PATH)
    #will search on google
    driver.get('http://www.google.com')
    search = driver.find_element_by_name('q')
    random_i = randint(5, 15)
    driver.implicitly_wait(random_i)
    # Enter search 
    search.send_keys(f'{subject} site:{website}')
    # hit return after you enter search text
    search.send_keys(Keys.RETURN) 
    get_hundred_results = str(driver.current_url)
    temporary_url = get_hundred_results + '&num=100'
    driver.get(temporary_url)
    return driver

def enter_date(driver, start_date, end_date):
    from random import randint
    # Find the tool bar
    element = driver.find_element_by_id('hdtb-tls')
    driver.execute_script("arguments[0].click();", element)
    #sleep
    random_i = randint(2, 3)
    driver.implicitly_wait(random_i)
    # Click on the date bar
    element2 = driver.find_element_by_class_name('hdtb-mn-hd')
    driver.execute_script("arguments[0].click();", element2)
    # Find the Customer Range menu
    element3 = driver.find_element_by_class_name('ErsxPb.gvybPb')
    driver.execute_script("arguments[0].click();", element3)
    #sleep
#     random_i = randint(2, 3)
#     driver.implicitly_wait(random_i)
    # Click the Customer Range menu
    element4 = WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.XPATH, '//*[@id="lb"]/div[2]/g-menu/g-menu-item[7]/div/div/span')))
    driver.execute_script("arguments[0].click();", element4)
    # Input Start Date
    element5 = driver.find_element_by_xpath('//*[@id="OouJcb"]')
    element5.send_keys(start_date)
    # Input End Date
    element6 = driver.find_element_by_xpath('//*[@id="rzG2be"]')
    element6.send_keys(end_date)
    # Random Sleep
    random_i = randint(2, 3)
    driver.implicitly_wait(random_i)
    # Submit Date
    element7 = driver.find_element_by_xpath('//*[@id="T3kYXe"]/g-button')
    driver.execute_script("arguments[0].click();", element7)
    return driver

def find_hrefs(driver, list_articles):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    random_i = randint(2, 5)
    driver.implicitly_wait(random_i)
    results = driver.find_elements_by_css_selector('div.g')
    for i in range(len(results)):
        random_i = randint(5, 15)
        driver.implicitly_wait(random_i)
        link = results[i].find_element_by_tag_name("a")
        href = link.get_attribute("href")
        list_articles.append(href)
    return list_articles, driver

def next_page(driver):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    try:
        random_i = randint(5, 15)
        time.sleep(random_i)
        next_page = driver.find_element_by_xpath("//*[@id='pnnext']").get_attribute("href")
        random_i = randint(5, 15)
        driver.implicitly_wait(random_i)
        driver.get(next_page)
        no_more_pages = False
        return no_more_pages
    
    except:
        print("no more pages left")
        no_more_pages = True
        return no_more_pages
    
  

# Calling the Functions

#### How to scrape hrefs

##### Selenium
- Use Selenium to google search each week within the desired timeframe. 
- Click through to page 2 and then quit, so each unique url can be gerated to include page tag.
- Create a list of 54 urls

##### Beautiful Soup
- Use Beautiful Soup to loop through each page and return the hrefs for every google page.
- For loop through each url until no more, then exit and continue on next url.

# Dailymail

#### Selenium

In [15]:
# https://www.reddit.com/r/webscraping/comments/iegnoo/scraping_100_google_search_results/

In [16]:
websites = ['dailymail.co.uk']

In [17]:
google_urls = []

In [21]:
import time 
from random import randint

i = 8
for website in websites:
    for x in range(12):
        search_result = search_selenium('nhs', website)
        random_i = randint(2, 5)
        time.sleep(random_i)
        start_date = search_days[i]
        print(start_date)
        end_date = search_days[i + 4]
        print(end_date)
        i += 4
        dated_website = enter_date(search_result, start_date, end_date)
        google_page = dated_website.current_url
        google_urls.append(google_page)



In [14]:
final_urls = []

In [15]:
i = 0
for x in range(12):
    start_date = search_days[i]
    end_date = search_days[i + 4]
    url = f'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{start_date}%2Ccd_max%3A{end_date}&tbm='
    final_urls.append(url)
    i += 4

In [16]:
final_urls

['https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A09-22-2019%2Ccd_max%3A10-20-2019&tbm=',
 'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10-20-2019%2Ccd_max%3A11-17-2019&tbm=',
 'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A11-17-2019%2Ccd_max%3A12-15-2019&tbm=',
 'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A12-15-2019%2Ccd_max%3A01-12-2020&tbm=',
 'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A01-12-2020%2Ccd_max%3A02-09-2020&tbm=',
 'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A02-09-2020%2Ccd_max%3A03-08-2020&tbm=',
 'https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A03-08-2020%2Ccd_max%3A04-05-2020&tbm=',

In [47]:
def search_url(url):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    #will search on google
    return driver

def find_hrefs(driver, list_articles):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    random_i = randint(60, 150)
    driver.implicitly_wait(random_i)
    results = driver.find_elements_by_css_selector('div.g')
    for i in range(len(results)):
        random_i = randint(30, 60)
        driver.implicitly_wait(random_i)
        link = results[i].find_element_by_tag_name("a")
        href = link.get_attribute("href")
        list_articles.append(href)
    return list_articles, driver
    
  

In [48]:
daily_hrefs = []

In [49]:
url =" https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10-20-2019%2Ccd_max%3A11-17-2019&tbm="
driver = search_url(url)
find_hrefs(driver, daily_hrefs)

([],
 <selenium.webdriver.chrome.webdriver.WebDriver (session="6d14098beb5d4e29f9e6af39867cca0f")>)

In [18]:
daily_hrefs = []
for url in final_urls:
    driver = search_url(url)
    no_more_pages = False
    while no_more_pages == False:
        random_i = randint(5, 15)
        time.sleep(random_i)
        temporary_url, no_more_pages = next_page(driver)
        google_searches.append(temporary_url)

no more pages left


TypeError: cannot unpack non-iterable bool object

In [22]:
google_searches = ['https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:09-22-2019,cd_max:10-20-2019&sxsrf=ALeKk03ydM1uOlbmbacECQDfKfRblZOmvA:1615231592886&ei=aHpGYIrSNcLO5gKup7mICQ&start=100&sa=N&ved=2ahUKEwiKpsvKtqHvAhVCp1kKHa5TDpEQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:10-20-2019,cd_max:11-17-2019&sxsrf=ALeKk03upYgBgeBVGYkP5v8pJKssXvKgHQ:1615231646483&ei=nnpGYKWEHdKw5wLRq4uYDA&start=100&sa=N&ved=2ahUKEwily5LktqHvAhVS2FkKHdHVAsMQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:11-17-2019,cd_max:12-15-2019&sxsrf=ALeKk03g7a2wAKvt1aWtDq9KZ5_OHm3heA:1615231669326&ei=tXpGYIS8E9bj-gTF1auwDg&start=100&sa=N&ved=2ahUKEwjE6oTvtqHvAhXWsZ4KHcXqCuYQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:07-26-2020,cd_max:08-23-2020&sxsrf=ALeKk03dW9BeDi6rDBHs5Hs1fwxjidrtHg:1615231860072&ei=dHtGYIvYA8G21fAP7P-64A0&start=100&sa=N&ved=2ahUKEwiL4v7Jt6HvAhVBWxUIHey_DtwQ8tMDegQIAxA1&biw=1280&bih=614','https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:06-28-2020,cd_max:07-26-2020&sxsrf=ALeKk01L6pumbSfTr9UojwIsMSCWLmEa6Q:1615231834750&ei=WntGYO2jLdj1-gSb6aTABw&start=100&sa=N&ved=2ahUKEwjtuPW9t6HvAhXYup4KHZs0CXgQ8tMDegQIAxA1&biw=1280&bih=614','https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:05-31-2020,cd_max:06-28-2020&sxsrf=ALeKk02U2nZAzNl8m3UBKkUKJpysuuaIgw:1615231782900&ei=JntGYKGtNu7v5gLQxZCIBg&start=100&sa=N&ved=2ahUKEwih2Jilt6HvAhXut1kKHdAiBGEQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:05-03-2020,cd_max:05-31-2020&sxsrf=ALeKk033iTODljozg1jnQwrhBbdgFpiU_g:1615231776327&ei=IHtGYOXAE8PI5gKsrpbAAg&start=100&sa=N&ved=2ahUKEwjl0Ieit6HvAhVDpFkKHSyXBSgQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:04-05-2020,cd_max:05-03-2020&sxsrf=ALeKk02OpHICcKx_nWof32_Ym-zrpitJOQ:1615231767079&ei=F3tGYMyuBIPX5gKKyLWgBw&start=100&sa=N&ved=2ahUKEwiMltOdt6HvAhWDq1kKHQpkDXQQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:03-08-2020,cd_max:04-05-2020&sxsrf=ALeKk01glPmRv1unHskyn3vwfBT3jXviZQ:1615231760208&ei=EHtGYKGhDKLO5gKdr6rwAQ&start=100&sa=N&ved=2ahUKEwih6a-at6HvAhUip1kKHZ2XCh4Q8tMDegQIAxA1&biw=1280&bih=614','https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:02-09-2020,cd_max:03-08-2020&sxsrf=ALeKk01ciPD8MK_FeNBmzRpPCSkBGgVVKA:1615231810604&ei=QntGYI-oJOGp5wL7vbTwAQ&start=100&sa=N&ved=2ahUKEwiP0bOyt6HvAhXh1FkKHfseDR4Q8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:01-12-2020,cd_max:02-09-2020&sxsrf=ALeKk02-3iH8dmQkdPtESgoeBFsgosJGNQ:1615231741484&ei=_XpGYLiJHY7a5gKnuo3oCw&start=100&sa=N&ved=2ahUKEwj4-7iRt6HvAhUOrVkKHSddA70Q8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:12-15-2019,cd_max:01-12-2020&sxsrf=ALeKk00vWfOeLL7tHORdEDrYTVC6Ezn0Ow:1615231723020&ei=63pGYJhmyabmAu33iogI&start=100&sa=N&ved=2ahUKEwjYh9KIt6HvAhVJk1kKHe27AoEQ8tMDegQIAxA1&biw=1280&bih=614']

In [46]:
google_searches

['https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:09-22-2019,cd_max:10-20-2019&sxsrf=ALeKk03ydM1uOlbmbacECQDfKfRblZOmvA:1615231592886&ei=aHpGYIrSNcLO5gKup7mICQ&start=100&sa=N&ved=2ahUKEwiKpsvKtqHvAhVCp1kKHa5TDpEQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:10-20-2019,cd_max:11-17-2019&sxsrf=ALeKk03upYgBgeBVGYkP5v8pJKssXvKgHQ:1615231646483&ei=nnpGYKWEHdKw5wLRq4uYDA&start=100&sa=N&ved=2ahUKEwily5LktqHvAhVS2FkKHdHVAsMQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:11-17-2019,cd_max:12-15-2019&sxsrf=ALeKk03g7a2wAKvt1aWtDq9KZ5_OHm3heA:1615231669326&ei=tXpGYIS8E9bj-gTF1auwDg&start=100&sa=N&ved=2ahUKEwjE6oTvtqHvAhXWsZ4KHcXqCuYQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:07-26-2020,cd_max:08-23-2020&sxsrf=ALeKk03dW9BeDi6rDBHs5Hs1fwxjidrtHg:161523186007

#### Beautiful Soup

In [24]:
# For loop through urls and generate all urls

In [25]:
start_ = []
for i in range(0, 400, 100):
    start_.append(f'start={i}')
start_

['start=0', 'start=100', 'start=200', 'start=300']

In [26]:
final_urls_list = []
for url in google_searches:    
    first_url = url.split('start=100')
    for i in range(len(start_)):
        final_urls_list.append(start_[i].join(first_url))
            

In [30]:
# Scrape hrefs google searches

In [50]:
# from bs4 import BeautifulSoup 
# import requests 
# import re 
# import time 

# for link in automated_list:
#     URL = link
#     headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
#     try:
#         resp = requests.get(URL, headers=headers).text 
#         random_i = randint(60, 150)
#         time.sleep(random_i)
#         print('sleep over')
#         soup = BeautifulSoup(resp, 'html.parser')
#         pattern = '^https://www.dailymail.co.uk/'
#         for link in soup.findAll('a', href=True): 
#             if link.get('href').startswith('https://www.dailymail.co.uk/'):
#                 print(link.get('href'))
#                 daily_mail_hrefs.append(link.get('href'))


#     except:
#         resp = None

# Beautiful Soup - Headlines and Straplines

In [51]:
import pandas as pd

In [ ]:
https://www.extendoffice.com/documents/excel/859-excel-list-hyperlinks.html

In [54]:
daily_mail_refs = pd.read_excel('C:/Users/jw156/Documents/all_links.xlsm')

In [57]:
all_daily_mail_refs = daily_mail_refs['Hyperlink Target'].tolist()

In [67]:
set_hrefs_mail = list(set(all_daily_mail_refs))

In [82]:
set_hrefs_mail

['https://discountcode.dailymail.co.uk/pooch-mutt',
 'https://www.dailymail.co.uk/news/article-8179995/73-NHS-workers-Londons-Great-Ormond-Street-Hospital-test-positive-coronavirus.html',
 'https://www.dailymail.co.uk/femail/article-7952031/Gran-claims-shes-selling-wheelchair-eBay-painful-arthritis-CURED-copper-insoles.html',
 'https://www.dailymail.co.uk/health/article-7804695/World-cancer-treatment-NHS-hospital-make-chemo-effective-using-BUBBLES.html',
 'https://www.dailymail.co.uk/health/article-8448393/Can-trust-antibody-test-expert-argues-theres-little-proof-work.html',
 'https://www.dailymail.co.uk/health/article-7515235/Give-ADULT-cancer-fighting-HPV-jab-save-thousands-lives-experts-demand.html',
 'https://www.dailymail.co.uk/news/article-8643457/Almost-75-hospital-Covid-19-patients-suffer-symptoms-three-months-later-study-claims.html',
 'https://www.dailymail.co.uk/news/article-7549921/Mother-said-aborted-son-knew-Downs-Syndrome-wins-NHS-payout.html',
 'https://www.dailymail.co

In [143]:
# article_text = []
dates = []
headline = []
import requests
from bs4 import BeautifulSoup
import time
for url in set_hrefs_mail[3146:]:
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
#     random_i = randint(5, 10)
#     time.sleep(random_i)
#     find the headlines
    try:
        article_headline = soup.find('h2').text
        print(article_headline)
        headline.append(article_headline)
    except:
        headline.append("No headline")
    # find the straplines
#     x = soup.find_all('ul', {'class': 'mol-bullets-with-font'})
#     if len(x) == 0:
#         article_text.append('No strapline')
#     else:
#         for ultag in soup.find_all('ul', {'class': 'mol-bullets-with-font'}):
#             local_list = []
#             for litag in ultag.find_all('li'):
#                     text = litag.text
#                     local_list.append(text)
#             article_text.append(local_list)
        

    try:
        date_time = soup.find('time').text
        article_date = date_time.split(',')[1]
        dates.append(article_date)
    except:
        dates.append("No date given")
        
#     print(f'Article text count: {len(article_text)}')
    print(f'Headline count: {len(headline)}')
    print(f'Dates count: {len(dates)}')

    print("-------------------------------------------------")

https://www.dailymail.co.uk/health/article-7902251/Doctors-fear-immunotherapy-actually-make-cancer-WORSE.html
Doctors fear immunotherapy could actually make cancer WORSE: For many the treatment is a lifeline, but one airport worker, 31, believes it has handed him a death sentence
Headline count: 1
Dates count: 1
-------------------------------------------------
https://www.dailymail.co.uk/tvshowbiz/article-8191037/James-Arthur-shaves-head-donates-5k-NHS-amid-coronavirus-pandemic.html
James Arthur jokes he's 'so happy hats were invented' as he shaves his head and donates £5k to the NHS amid the coronavirus pandemic
Headline count: 2
Dates count: 2
-------------------------------------------------
https://www.dailymail.co.uk/news/article-8220899/Duke-Westminster-donates-12-5million-NHS-coronavirus-fight.html
The Duke of Westminster gives £12.5m to the NHS: Prince George's billionaire godfather, 29, makes Britain's biggest personal donation to fight coronavirus after being 'humbled' by fr

Starbucks is rapped for launching 'woke' campaign to raise £100,000 for controversial transgender charity
Headline count: 25
Dates count: 25
-------------------------------------------------
https://www.dailymail.co.uk/news/article-7630109/Father-woman-died-overdosing-diet-pills-blasts-pathetic-Government-response.html
Father of woman, 21, who died after overdosing on diet pills blasts 'utterly pathetic' Government for not doing more to stop supply of poisonous drug DHP
Headline count: 26
Dates count: 26
-------------------------------------------------
https://www.dailymail.co.uk/news/article-8367001/Prince-William-warns-hidden-mental-health-dangers-praising-NHS-workers.html
'Even superheroes need support': Prince William warns of the 'hidden mental health dangers' of praising NHS workers amid fears they won't ask for help if they are struggling
Headline count: 27
Dates count: 27
-------------------------------------------------
https://www.dailymail.co.uk/tvshowbiz/article-8377717/Dr

Netflix will end service on older Roku models by December because it will not compatible with new streaming app
Headline count: 49
Dates count: 49
-------------------------------------------------
https://www.dailymail.co.uk/sport/football/article-8556071/Liverpool-captain-Jordan-Henderson-beats-Kevin-Bruyne-win-FWA-Footballer-Year.html
Jordan Henderson beats Kevin De Bruyne and his Liverpool team-mates to the FWA Footballer of the Year award as the title-winning captain is rewarded for his outstanding year ON AND OFF the pitch
Headline count: 50
Dates count: 50
-------------------------------------------------
https://www.dailymail.co.uk/health/article-7524907/Could-eating-meat-REALLY-cut-risk-cancer-heart-disease-type-2-diabetes.html
Could eating less meat REALLY cut your risk of cancer, heart disease and type 2 diabetes? As researchers say you DON'T need to cut out beef... we reveal what they uncovered about having three fewer portions each week
Headline count: 51
Dates count: 51
--

We need an evacuation plan now: Lib Dem leader Ed Davey says UK must prepare to bring British citizens and diplomats back from Wuhan after US sends plane to rescue 1,000 Americans trapped in coronavirus hotspot
Headline count: 72
Dates count: 72
-------------------------------------------------
https://www.dailymail.co.uk/health/article-7864729/22-glaucoma-patients-England-predicted-blind-month-delays-NHS-appointments.html
NHS patients are being left to go BLIND because of year-long delays in getting appointments for glaucoma and other vision-robbing conditions, damning report warns
Headline count: 73
Dates count: 73
-------------------------------------------------
https://www.dailymail.co.uk/health/article-7621221/Obese-PRETEENS-offered-weight-loss-surgery-paediatricians-say.html
Severely obese children as young as TWELVE should be offered gastric bypasses 'because it is safe and effective', paediatricians say
Headline count: 74
Dates count: 74
---------------------------------------

Half of all schoolchildren in England don't take part in the recommended one hour of exercise each day, poll reveals
Headline count: 96
Dates count: 96
-------------------------------------------------
https://discountcode.dailymail.co.uk/penhaligon
Save with 12 Penhaligon voucher codes and offers
Headline count: 97
Dates count: 97
-------------------------------------------------
https://www.dailymail.co.uk/news/article-8362567/Nurse-slammed-yelling-carer-wearing-scrubs-shop-backed-nursing-body.html
ICU nurse slammed for yelling at crying carer for wearing her uniform in a B&M store is BACKED by Royal College of Nursing who say shopping in work clothes ‘should be avoided’
Headline count: 98
Dates count: 98
-------------------------------------------------
https://www.dailymail.co.uk/news/article-8568891/NHS-worker-suffers-horrific-injuries-racist-hit-run-attack-yobs.html
Black NHS worker suffers horrific injuries in 'racist hit-and-run attack' by yobs who yelled 'f***ing n*****' as th

In [ ]:
# article_text

In [145]:
#initialize dataframe
daily_mail_df_2 = pd.DataFrame()
#populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
daily_mail_df_2['date'] = [d for d in dates]
daily_mail_df_2['headline'] = [line for line in headline]
daily_mail_df_2['weblink'] = [ref for ref in set_hrefs_mail[3146:]]

In [148]:
daily_mail_final = pd.concat([daily_mail_df_2, daily_mail_df_1], axis=0, ignore_index=True)

In [149]:
daily_mail_final

,date,headline,weblink
0,18 January 2020,Doctors fear immunotherapy could actually make...,https://www.dailymail.co.uk/health/article-790...
1,6 April 2020,James Arthur jokes he's 'so happy hats were in...,https://www.dailymail.co.uk/tvshowbiz/article-...
2,15 April 2020,The Duke of Westminster gives £12.5m to the NH...,https://www.dailymail.co.uk/news/article-82208...
3,25 July 2020,Covid crisis is 'destroying women's careers' a...,https://www.dailymail.co.uk/news/article-85586...
4,16 December 2019,"Student, 21, battles sepsis after she caught a...",https://www.dailymail.co.uk/health/article-779...
...,...,...,...
3246,9 December 2019,Thrifty father-of-three reveals how he saves £...,https://www.dailymail.co.uk/femail/article-773...
3247,20 December 2019,The Cure viewers brand the show a 'heartbreaki...,https://www.dailymail.co.uk/femail/article-781...
3248,26 March 2020,SARAH VINE: When we look back on these extraor...,https://www.dailymail.co.uk/debate/article-815...
3249,25 September 2019,Men who struggle to become fathers are up to 6...,https://www.dailymail.co.uk/health/article-750...


In [150]:
daily_mail_df_1=daily_mail_df
daily_mail_final.to_csv("C:/Users/jw156/Ironhack/Final_Project/NHS_headlines/scrape/daily_mail_final_df.csv")
print("Sucessfully saFed")

Sucessfully saFed


In [141]:
daily_mail_df_1

,date,headline,weblink
0,No date given,Save with 18 Pooch and Mutt coupons and offers,https://discountcode.dailymail.co.uk/pooch-mutt
1,2 April 2020,73 NHS workers at London’s Great Ormond Street...,https://www.dailymail.co.uk/news/article-81799...
2,31 January 2020,"Grandmother, 75, who claims her crippling arth...",https://www.dailymail.co.uk/femail/article-795...
3,18 December 2019,World first cancer treatment being trialled in...,https://www.dailymail.co.uk/health/article-780...
4,22 June 2020,Can we trust ANY antibody test? With more than...,https://www.dailymail.co.uk/health/article-844...
...,...,...,...
3141,9 December 2019,Thrifty father-of-three reveals how he saves £...,https://www.dailymail.co.uk/femail/article-773...
3142,20 December 2019,The Cure viewers brand the show a 'heartbreaki...,https://www.dailymail.co.uk/femail/article-781...
3143,26 March 2020,SARAH VINE: When we look back on these extraor...,https://www.dailymail.co.uk/debate/article-815...
3144,25 September 2019,Men who struggle to become fathers are up to 6...,https://www.dailymail.co.uk/health/article-750...


In [ ]:
# with open("daily_mail_article_text.txt", "w") as output:
#     for s in article_text:
#         outfile.write("%s\n" % s)

#     output.write(str(article_text))

# with open("daily_mail_dates.txt", "w") as output:
#     for s in dates:
#         outfile.write("%s\n" % s)

# with open("daily_mail_headline.txt", "w") as output:
#     for s in headline:
#         outfile.write("%s\n" % s)

# Telegraph

In [24]:
telegraph_links = ['https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:09-22-2019,cd_max:10-20-2019&sxsrf=ALeKk03ydM1uOlbmbacECQDfKfRblZOmvA:1615231592886&ei=aHpGYIrSNcLO5gKup7mICQ&start=100&sa=N&ved=2ahUKEwiKpsvKtqHvAhVCp1kKHa5TDpEQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk.co.uk&num=100&tbs=cdr:1,cd_min:10-20-2019,cd_max:11-17-2019&sxsrf=ALeKk03upYgBgeBVGYkP5v8pJKssXvKgHQ:1615231646483&ei=nnpGYKWEHdKw5wLRq4uYDA&start=100&sa=N&ved=2ahUKEwily5LktqHvAhVS2FkKHdHVAsMQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:11-17-2019,cd_max:12-15-2019&sxsrf=ALeKk03g7a2wAKvt1aWtDq9KZ5_OHm3heA:1615231669326&ei=tXpGYIS8E9bj-gTF1auwDg&start=100&sa=N&ved=2ahUKEwjE6oTvtqHvAhXWsZ4KHcXqCuYQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:07-26-2020,cd_max:08-23-2020&sxsrf=ALeKk03dW9BeDi6rDBHs5Hs1fwxjidrtHg:1615231860072&ei=dHtGYIvYA8G21fAP7P-64A0&start=100&sa=N&ved=2ahUKEwiL4v7Jt6HvAhVBWxUIHey_DtwQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:06-28-2020,cd_max:07-26-2020&sxsrf=ALeKk01L6pumbSfTr9UojwIsMSCWLmEa6Q:1615231834750&ei=WntGYO2jLdj1-gSb6aTABw&start=100&sa=N&ved=2ahUKEwjtuPW9t6HvAhXYup4KHZs0CXgQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:05-31-2020,cd_max:06-28-2020&sxsrf=ALeKk02U2nZAzNl8m3UBKkUKJpysuuaIgw:1615231782900&ei=JntGYKGtNu7v5gLQxZCIBg&start=100&sa=N&ved=2ahUKEwih2Jilt6HvAhXut1kKHdAiBGEQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:05-03-2020,cd_max:05-31-2020&sxsrf=ALeKk033iTODljozg1jnQwrhBbdgFpiU_g:1615231776327&ei=IHtGYOXAE8PI5gKsrpbAAg&start=100&sa=N&ved=2ahUKEwjl0Ieit6HvAhVDpFkKHSyXBSgQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:04-05-2020,cd_max:05-03-2020&sxsrf=ALeKk02OpHICcKx_nWof32_Ym-zrpitJOQ:1615231767079&ei=F3tGYMyuBIPX5gKKyLWgBw&start=100&sa=N&ved=2ahUKEwiMltOdt6HvAhWDq1kKHQpkDXQQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:03-08-2020,cd_max:04-05-2020&sxsrf=ALeKk01glPmRv1unHskyn3vwfBT3jXviZQ:1615231760208&ei=EHtGYKGhDKLO5gKdr6rwAQ&start=100&sa=N&ved=2ahUKEwih6a-at6HvAhUip1kKHZ2XCh4Q8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:02-09-2020,cd_max:03-08-2020&sxsrf=ALeKk01ciPD8MK_FeNBmzRpPCSkBGgVVKA:1615231810604&ei=QntGYI-oJOGp5wL7vbTwAQ&start=100&sa=N&ved=2ahUKEwiP0bOyt6HvAhXh1FkKHfseDR4Q8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:01-12-2020,cd_max:02-09-2020&sxsrf=ALeKk02-3iH8dmQkdPtESgoeBFsgosJGNQ:1615231741484&ei=_XpGYLiJHY7a5gKnuo3oCw&start=100&sa=N&ved=2ahUKEwj4-7iRt6HvAhUOrVkKHSddA70Q8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:12-15-2019,cd_max:01-12-2020&sxsrf=ALeKk00vWfOeLL7tHORdEDrYTVC6Ezn0Ow:1615231723020&ei=63pGYJhmyabmAu33iogI&start=100&sa=N&ved=2ahUKEwjYh9KIt6HvAhVJk1kKHe27AoEQ8tMDegQIAxA1&biw=1280&bih=614']


In [25]:
telegraph_links

['https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:09-22-2019,cd_max:10-20-2019&sxsrf=ALeKk03ydM1uOlbmbacECQDfKfRblZOmvA:1615231592886&ei=aHpGYIrSNcLO5gKup7mICQ&start=100&sa=N&ved=2ahUKEwiKpsvKtqHvAhVCp1kKHa5TDpEQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk.co.uk&num=100&tbs=cdr:1,cd_min:10-20-2019,cd_max:11-17-2019&sxsrf=ALeKk03upYgBgeBVGYkP5v8pJKssXvKgHQ:1615231646483&ei=nnpGYKWEHdKw5wLRq4uYDA&start=100&sa=N&ved=2ahUKEwily5LktqHvAhVS2FkKHdHVAsMQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:11-17-2019,cd_max:12-15-2019&sxsrf=ALeKk03g7a2wAKvt1aWtDq9KZ5_OHm3heA:1615231669326&ei=tXpGYIS8E9bj-gTF1auwDg&start=100&sa=N&ved=2ahUKEwjE6oTvtqHvAhXWsZ4KHcXqCuYQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:07-26-2020,cd_max:08-23-2020&sxsrf=ALeKk03dW9BeDi6rDBHs5Hs1fwxjidrtHg:161523

#### Selenium

In [1]:
final_urls = []

In [20]:
i = 0
for x in range(12):
    start_date = search_days[i]
    end_date = search_days[i + 4]
    a = ':'
    url = f'https://www.google.com/search?q=nhs+site{a}telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A{start_date}%2Ccd_max%3A{end_date}&tbm='
    final_urls.append(url)
    i += 4

In [23]:
final_urls

['https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A09-22-2019%2Ccd_max%3A10-20-2019&tbm=',
 'https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10-20-2019%2Ccd_max%3A11-17-2019&tbm=',
 'https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A11-17-2019%2Ccd_max%3A12-15-2019&tbm=',
 'https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A12-15-2019%2Ccd_max%3A01-12-2020&tbm=',
 'https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A01-12-2020%2Ccd_max%3A02-09-2020&tbm=',
 'https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A02-09-2020%2Ccd_max%3A03-08-2020&tbm=',
 'https://www.google.com/search?q=nhs+site%telegraph.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A03-08-2020%2Ccd_max%3A04-05-2020&tbm=',
 'https://www

In [47]:
def search_url(url):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    driver = webdriver.Chrome(PATH)
    driver.get(url)
    #will search on google
    return driver

def find_hrefs(driver, list_articles):
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.support import expected_conditions as EC
    from random import randint
    random_i = randint(60, 150)
    driver.implicitly_wait(random_i)
    results = driver.find_elements_by_css_selector('div.g')
    for i in range(len(results)):
        random_i = randint(30, 60)
        driver.implicitly_wait(random_i)
        link = results[i].find_element_by_tag_name("a")
        href = link.get_attribute("href")
        list_articles.append(href)
    return list_articles, driver
    
  

In [48]:
daily_hrefs = []

In [49]:
url =" https://www.google.com/search?q=nhs+site%3Adailymail.co.uk&num=100&source=lnt&tbs=cdr%3A1%2Ccd_min%3A10-20-2019%2Ccd_max%3A11-17-2019&tbm="
driver = search_url(url)
find_hrefs(driver, daily_hrefs)

([],
 <selenium.webdriver.chrome.webdriver.WebDriver (session="6d14098beb5d4e29f9e6af39867cca0f")>)

In [18]:
daily_hrefs = []
for url in final_urls:
    driver = search_url(url)
    no_more_pages = False
    while no_more_pages == False:
        random_i = randint(5, 15)
        time.sleep(random_i)
        temporary_url, no_more_pages = next_page(driver)
        google_searches.append(temporary_url)

no more pages left


TypeError: cannot unpack non-iterable bool object

In [13]:
google_searches = ['https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:09-22-2019,cd_max:10-20-2019&sxsrf=ALeKk03ydM1uOlbmbacECQDfKfRblZOmvA:1615231592886&ei=aHpGYIrSNcLO5gKup7mICQ&start=100&sa=N&ved=2ahUKEwiKpsvKtqHvAhVCp1kKHa5TDpEQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:10-20-2019,cd_max:11-17-2019&sxsrf=ALeKk03upYgBgeBVGYkP5v8pJKssXvKgHQ:1615231646483&ei=nnpGYKWEHdKw5wLRq4uYDA&start=100&sa=N&ved=2ahUKEwily5LktqHvAhVS2FkKHdHVAsMQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:11-17-2019,cd_max:12-15-2019&sxsrf=ALeKk03g7a2wAKvt1aWtDq9KZ5_OHm3heA:1615231669326&ei=tXpGYIS8E9bj-gTF1auwDg&start=100&sa=N&ved=2ahUKEwjE6oTvtqHvAhXWsZ4KHcXqCuYQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:07-26-2020,cd_max:08-23-2020&sxsrf=ALeKk03dW9BeDi6rDBHs5Hs1fwxjidrtHg:1615231860072&ei=dHtGYIvYA8G21fAP7P-64A0&start=100&sa=N&ved=2ahUKEwiL4v7Jt6HvAhVBWxUIHey_DtwQ8tMDegQIAxA1&biw=1280&bih=614','https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:06-28-2020,cd_max:07-26-2020&sxsrf=ALeKk01L6pumbSfTr9UojwIsMSCWLmEa6Q:1615231834750&ei=WntGYO2jLdj1-gSb6aTABw&start=100&sa=N&ved=2ahUKEwjtuPW9t6HvAhXYup4KHZs0CXgQ8tMDegQIAxA1&biw=1280&bih=614','https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:05-31-2020,cd_max:06-28-2020&sxsrf=ALeKk02U2nZAzNl8m3UBKkUKJpysuuaIgw:1615231782900&ei=JntGYKGtNu7v5gLQxZCIBg&start=100&sa=N&ved=2ahUKEwih2Jilt6HvAhXut1kKHdAiBGEQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:05-03-2020,cd_max:05-31-2020&sxsrf=ALeKk033iTODljozg1jnQwrhBbdgFpiU_g:1615231776327&ei=IHtGYOXAE8PI5gKsrpbAAg&start=100&sa=N&ved=2ahUKEwjl0Ieit6HvAhVDpFkKHSyXBSgQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:04-05-2020,cd_max:05-03-2020&sxsrf=ALeKk02OpHICcKx_nWof32_Ym-zrpitJOQ:1615231767079&ei=F3tGYMyuBIPX5gKKyLWgBw&start=100&sa=N&ved=2ahUKEwiMltOdt6HvAhWDq1kKHQpkDXQQ8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:03-08-2020,cd_max:04-05-2020&sxsrf=ALeKk01glPmRv1unHskyn3vwfBT3jXviZQ:1615231760208&ei=EHtGYKGhDKLO5gKdr6rwAQ&start=100&sa=N&ved=2ahUKEwih6a-at6HvAhUip1kKHZ2XCh4Q8tMDegQIAxA1&biw=1280&bih=614','https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:02-09-2020,cd_max:03-08-2020&sxsrf=ALeKk01ciPD8MK_FeNBmzRpPCSkBGgVVKA:1615231810604&ei=QntGYI-oJOGp5wL7vbTwAQ&start=100&sa=N&ved=2ahUKEwiP0bOyt6HvAhXh1FkKHfseDR4Q8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:01-12-2020,cd_max:02-09-2020&sxsrf=ALeKk02-3iH8dmQkdPtESgoeBFsgosJGNQ:1615231741484&ei=_XpGYLiJHY7a5gKnuo3oCw&start=100&sa=N&ved=2ahUKEwj4-7iRt6HvAhUOrVkKHSddA70Q8tMDegQIAxA1&biw=1280&bih=614', 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:12-15-2019,cd_max:01-12-2020&sxsrf=ALeKk00vWfOeLL7tHORdEDrYTVC6Ezn0Ow:1615231723020&ei=63pGYJhmyabmAu33iogI&start=100&sa=N&ved=2ahUKEwjYh9KIt6HvAhVJk1kKHe27AoEQ8tMDegQIAxA1&biw=1280&bih=614']

In [14]:
google_searches

['https://www.google.com/search?q=nhs+site:telegraph.co.uk&num=100&tbs=cdr:1,cd_min:09-22-2019,cd_max:10-20-2019&sxsrf=ALeKk03ydM1uOlbmbacECQDfKfRblZOmvA:1615231592886&ei=aHpGYIrSNcLO5gKup7mICQ&start=100&sa=N&ved=2ahUKEwiKpsvKtqHvAhVCp1kKHa5TDpEQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:10-20-2019,cd_max:11-17-2019&sxsrf=ALeKk03upYgBgeBVGYkP5v8pJKssXvKgHQ:1615231646483&ei=nnpGYKWEHdKw5wLRq4uYDA&start=100&sa=N&ved=2ahUKEwily5LktqHvAhVS2FkKHdHVAsMQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:11-17-2019,cd_max:12-15-2019&sxsrf=ALeKk03g7a2wAKvt1aWtDq9KZ5_OHm3heA:1615231669326&ei=tXpGYIS8E9bj-gTF1auwDg&start=100&sa=N&ved=2ahUKEwjE6oTvtqHvAhXWsZ4KHcXqCuYQ8tMDegQIAxA1&biw=1280&bih=614',
 'https://www.google.com/search?q=nhs+site:dailymail.co.uk&num=100&tbs=cdr:1,cd_min:07-26-2020,cd_max:08-23-2020&sxsrf=ALeKk03dW9BeDi6rDBHs5Hs1fwxjidrtHg:161523186007

#### Beautiful Soup

In [24]:
# For loop through urls and generate all urls

In [25]:
start_ = []
for i in range(0, 400, 100):
    start_.append(f'start={i}')
start_

['start=0', 'start=100', 'start=200', 'start=300']

In [26]:
final_urls_list = []
for url in google_searches:    
    first_url = url.split('start=100')
    for i in range(len(start_)):
        final_urls_list.append(start_[i].join(first_url))
            

In [30]:
# Scrape hrefs google searches

In [50]:
# from bs4 import BeautifulSoup 
# import requests 
# import re 
# import time 

# for link in automated_list:
#     URL = link
#     headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
#     try:
#         resp = requests.get(URL, headers=headers).text 
#         random_i = randint(60, 150)
#         time.sleep(random_i)
#         print('sleep over')
#         soup = BeautifulSoup(resp, 'html.parser')
#         pattern = '^https://www.dailymail.co.uk/'
#         for link in soup.findAll('a', href=True): 
#             if link.get('href').startswith('https://www.dailymail.co.uk/'):
#                 print(link.get('href'))
#                 daily_mail_hrefs.append(link.get('href'))


#     except:
#         resp = None

# Beautiful Soup - Headlines and Straplines

In [26]:
import pandas as pd

In [27]:
telegraph_refs = pd.read_excel('C:/Users/jw156/Documents/all_links.xlsm')

In [28]:
telegraph_links = telegraph_refs['Hyperlink Target'].tolist()

In [29]:
set_hrefs_telegraph = list(set(telegraph_links))

In [31]:
len(set_hrefs_telegraph)

3229

In [35]:
# article_text = []
dates = []
headline = []
import requests
from bs4 import BeautifulSoup
import time
for url in set_hrefs_telegraph:
    print(url)
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    random_i = randint(5, 10)
    time.sleep(random_i)
#     find the headlines
    try:
        article_headline = soup.find('h1').text
        print(article_headline)
        headline.append(article_headline)
    except:
        headline.append("No headline")
    # find the straplines
#     x = soup.find_all('ul', {'class': 'mol-bullets-with-font'})
#     if len(x) == 0:
#         article_text.append('No strapline')
#     else:
#         for ultag in soup.find_all('ul', {'class': 'mol-bullets-with-font'}):
#             local_list = []
#             for litag in ultag.find_all('li'):
#                     text = litag.text
#                     local_list.append(text)
#             article_text.append(local_list)
        

    try:
        date_time = soup.find('time').text
        article_date = date_time.split(',')[1]
        dates.append(article_date)
    except:
        dates.append("No date given")
        
#     print(f'Article text count: {len(article_text)}')
    print(f'Headline count: {len(headline)}')
    print(f'Dates count: {len(dates)}')

    print("-------------------------------------------------")

https://www.telegraph.co.uk/technology/2020/06/19/apple-google-contact-tracing-may-miss-95pc-matches-buses/

Apple and Google contact-tracing technology 'won't work on buses', research says

Headline count: 1
Dates count: 1
-------------------------------------------------
https://www.telegraph.co.uk/opinion/2020/02/03/lettersthe-bbc-revealed-contempt-nation-brexit-day-coverage/

Letters: The BBC revealed its contempt for the nation in its Brexit Day coverage

Headline count: 2
Dates count: 2
-------------------------------------------------
https://www.telegraph.co.uk/family/parenting/how-keep-babies-cool-summer-heat-heatwave/

How to keep babies and toddlers cool during a summer heatwave

Headline count: 3
Dates count: 3
-------------------------------------------------
https://www.telegraph.co.uk/business/2020/05/16/government-accused-wasting-taxpayers-money-paying-odds-ppe-china/

Government accused of wasting taxpayers’ money paying over odds for PPE from China

Headline count: 4



Coronavirus latest news: Surge in UK cases as total hits 87

Headline count: 33
Dates count: 33
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/01/14/pub-banter-social-media-posts-can-sexual-harassment-equality/

Pub 'banter' and social media posts can be sexual harassment, equality watchdog warns bosses

Headline count: 34
Dates count: 34
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/05/05/pictures-day-5th-may-2020/

Pictures of the Day: 5th May 2020

Headline count: 35
Dates count: 35
-------------------------------------------------
https://www.telegraph.co.uk/health-fitness/body/can-lose-five-pounds-like-boris-johnson/

Quick tips to help you lose 5lb like Boris Johnson

Headline count: 36
Dates count: 36
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/05/21/superdrug-coronavirus-antibody-test/

Superdrug accused of 'playing on people's fear' with £69 antibodie


Spanish regions ban smoking outdoors in bid to curb Covid transmission

Headline count: 64
Dates count: 64
-------------------------------------------------
https://www.telegraph.co.uk/global-health/science-and-disease/coronavirus-news-cases-deaths-lockdown-leicester/

200 workers quarantined at Herefordshire farm after 73 catch Covid-19

Headline count: 65
Dates count: 65
-------------------------------------------------
https://www.telegraph.co.uk/news/2019/10/03/extra-half-glass-fruit-juice-day-could-sharply-increase-diabetes/

An extra half glass of fruit juice a day could sharply increase diabetes risk

Headline count: 66
Dates count: 66
-------------------------------------------------
https://www.telegraph.co.uk/family/parenting/look-knees-plan-future-geriatric-parent-take-note-boris/

Look after your knees and plan for the future: how to be a geriatric parent (take note, Boris!)

Headline count: 67
Dates count: 67
-------------------------------------------------
https://www.t


Clap For Our Carers: How Britons thanked NHS with nationwide round of applause

Headline count: 95
Dates count: 95
-------------------------------------------------
https://www.telegraph.co.uk/business/business-reporter/damira-dental-studios/

Damira Dental Studios: best of British business

Headline count: 96
Dates count: 96
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/03/30/nurses-refusing-treat-critical-coronavirus-patients-lack-protective/

Nurses refusing to treat critical coronavirus patients because of lack of protective equipment, frontline NHS staff warn

Headline count: 97
Dates count: 97
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/07/16/covid-hospital-robots-could-hacked-cyber-terrorism/

Covid hospital robots 'could be hacked for cyber-terrorism'

Headline count: 98
Dates count: 98
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/02/22/mild-hearing-


Britain in coronavirus lockdown, in pictures

Headline count: 127
Dates count: 127
-------------------------------------------------
https://www.telegraph.co.uk/news/2019/10/19/university-considers-legal-action-against-mole-claiming-lecturers/

University considers legal action against mole claiming lecturers 'indoctrinate' students with 'far-left' ideology

Headline count: 128
Dates count: 128
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/05/04/challenging-lockdown-court/

Why I'm challenging lockdown in court

Headline count: 129
Dates count: 129
-------------------------------------------------
https://www.telegraph.co.uk/cars/comment/good-samaritan-delivery-drivers-may-not-seem/

Why 'good Samaritan' delivery drivers may not be what they seem

Headline count: 130
Dates count: 130
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/04/11/former-health-minister-lord-darzi-back-nhs-frontline/

Lord Darzi de

...


NHS chief attacks Gwyneth Paltrow's Goop as he warns against 'quacks, charlatans and cranks'

Headline count: 1229
Dates count: 1229
-------------------------------------------------
https://www.telegraph.co.uk/global-health/science-and-disease/coronavirus-news-contact-tracing-spain-second-wave-cases/

UK’s official death toll reduced by 5,377 following review of figures

Headline count: 1230
Dates count: 1230
-------------------------------------------------
https://www.telegraph.co.uk/news/2019/11/18/retired-scotland-yard-chief-tackled-ground-using-old-police/

Retired Scotland Yard chief tackled to the ground after using old police pass to dodge train fare, court hears

Headline count: 1231
Dates count: 1231
-------------------------------------------------
https://www.telegraph.co.uk/politics/2019/12/13/tories-win-general-election-happens-next-will-brexit-happen/

What next for Brexit after Conservatives win the general election?

Headline count: 1232
Dates count: 1232
-----------


Exhausted nurses at overwhelmed London hospital wore bin bags to protect themselves from coronavirus

Headline count: 1260
Dates count: 1260
-------------------------------------------------
https://www.telegraph.co.uk/politics/2020/05/09/thinking-behind-boris-johnsons-slogan-phase-two-covid-19-pandemic/

The thinking behind Boris Johnson's slogan for phase two of Covid-19 pandemic

Headline count: 1261
Dates count: 1261
-------------------------------------------------
https://www.telegraph.co.uk/politics/2020/03/12/wests-neglect-science-has-left-us-chillingly-exposed-deadly/

The West's neglect of science has left us chillingly exposed to deadly pandemics 

Headline count: 1262
Dates count: 1262
-------------------------------------------------
https://www.telegraph.co.uk/news/2020/04/21/did-bbc-come-report-slanted-sloppy-story/

How did the BBC come to report such a slanted, sloppy story?

Headline count: 1263
Dates count: 1263
-------------------------------------------------
http


A million elderly people skipping meals because they find eating alone too lonely, charity reveals

Headline count: 1291
Dates count: 1291
-------------------------------------------------
https://www.telegraph.co.uk/politics/2020/04/12/politics-latest-news-matt-hancock-gives-coronavirus-update-death/

Matt Hancock speaks of 'a sombre day'  with 10,000 lives now lost to coronavirus

Headline count: 1292
Dates count: 1292
-------------------------------------------------
https://www.telegraph.co.uk/technology/2020/06/18/london-has-lost-allure-uk-tech-start-ups/

London has lost its allure for UK tech start-ups

Headline count: 1293
Dates count: 1293
-------------------------------------------------
https://www.telegraph.co.uk/technology/2019/10/23/british-schools-test-robot-teachers-tackle-staff-shortages/

British schools test 'robot teachers' to tackle staff shortages

Headline count: 1294
Dates count: 1294
-------------------------------------------------
https://www.telegraph.co.uk


Exclusive: Test and trace strategy would currently fail to prevent a second wave, says top official

Headline count: 1321
Dates count: 1321
-------------------------------------------------
http://www.telegraph.co.uk/news/2020/07/10/coronavirus-antibody-test-where-kits-buy/


TooManyRedirects: Exceeded 30 redirects.

In [ ]:
# article_text

In [37]:
#initialize dataframe
telegraph_1 = pd.DataFrame()
#populate date and average temperature fields (cast string date to datetime and convert temperature from tenths of Celsius to Fahrenheit)
telegraph_1['date'] = [d for d in dates]
telegraph_1['headline'] = [line for line in headline]
telegraph_1['weblink'] = [ref for ref in set_hrefs_telegraph[1908:]]

In [148]:
daily_mail_final = pd.concat([daily_mail_df_2, daily_mail_df_1], axis=0, ignore_index=True)

In [149]:
daily_mail_final

,date,headline,weblink
0,18 January 2020,Doctors fear immunotherapy could actually make...,https://www.dailymail.co.uk/health/article-790...
1,6 April 2020,James Arthur jokes he's 'so happy hats were in...,https://www.dailymail.co.uk/tvshowbiz/article-...
2,15 April 2020,The Duke of Westminster gives £12.5m to the NH...,https://www.dailymail.co.uk/news/article-82208...
3,25 July 2020,Covid crisis is 'destroying women's careers' a...,https://www.dailymail.co.uk/news/article-85586...
4,16 December 2019,"Student, 21, battles sepsis after she caught a...",https://www.dailymail.co.uk/health/article-779...
...,...,...,...
3246,9 December 2019,Thrifty father-of-three reveals how he saves £...,https://www.dailymail.co.uk/femail/article-773...
3247,20 December 2019,The Cure viewers brand the show a 'heartbreaki...,https://www.dailymail.co.uk/femail/article-781...
3248,26 March 2020,SARAH VINE: When we look back on these extraor...,https://www.dailymail.co.uk/debate/article-815...
3249,25 September 2019,Men who struggle to become fathers are up to 6...,https://www.dailymail.co.uk/health/article-750...


In [38]:
telegraph_1=telegraph_1
telegraph_1.to_csv("C:/Users/jw156/Ironhack/Final_Project/NHS_headlines/scrape/telegraph_final_df.csv")
print("Sucessfully saFed")

Sucessfully saFed


In [141]:
daily_mail_df_1

,date,headline,weblink
0,No date given,Save with 18 Pooch and Mutt coupons and offers,https://discountcode.dailymail.co.uk/pooch-mutt
1,2 April 2020,73 NHS workers at London’s Great Ormond Street...,https://www.dailymail.co.uk/news/article-81799...
2,31 January 2020,"Grandmother, 75, who claims her crippling arth...",https://www.dailymail.co.uk/femail/article-795...
3,18 December 2019,World first cancer treatment being trialled in...,https://www.dailymail.co.uk/health/article-780...
4,22 June 2020,Can we trust ANY antibody test? With more than...,https://www.dailymail.co.uk/health/article-844...
...,...,...,...
3141,9 December 2019,Thrifty father-of-three reveals how he saves £...,https://www.dailymail.co.uk/femail/article-773...
3142,20 December 2019,The Cure viewers brand the show a 'heartbreaki...,https://www.dailymail.co.uk/femail/article-781...
3143,26 March 2020,SARAH VINE: When we look back on these extraor...,https://www.dailymail.co.uk/debate/article-815...
3144,25 September 2019,Men who struggle to become fathers are up to 6...,https://www.dailymail.co.uk/health/article-750...


# Scrap Paper

# Scrape Google with Beautiful Soup

In [83]:
# def scrape_daily_mail_headlines(list_urls):
#     article_text = []
#     dates = []
#     headline = []
#     import requests
#     from bs4 import BeautifulSoup
#     for i in range(len(set_hrefs_mail)):
#         print(i)
#         print(list_urls[i])
#         print("-------------------------------------------------")
#         req = requests.get(list_urls[i])
#         soup = BeautifulSoup(req.content, 'html.parser')
#         # sleep
#         random_i = randint(5, 10)
#         time.sleep(random_i)
#         # find the headlines
#         article_headline = soup.find('h2').text
#         headline.append(article_headline)
#         # find the straplines
#         for ultag in soup.find_all('ul', {'class': 'mol-bullets-with-font'}):
#             local_list = []
#             for litag in ultag.find_all('li'):
#                 text = litag.text
#                 local_list.append(text)
#             article_text.append(local_list)
            
#         # get the timestamp
#         try:
#             date_time = soup.find('time').text
#             article_date = date_time.split(',')[1]
#             dates.append(article_date)
#         except:
#             dates.append("No date given")

#     return article_text, dates, headline



In [89]:
# import requests
# from bs4 import BeautifulSoup


# url = 'https://www.google.com/search?sxsrf=ALeKk01NCXLLm0RJBab9tu4aUEXHyabIKQ%3A1614890859311&ei=a0dBYJnGEoSjgQaK4o4g&q=nhs+site%3Adailymail.co.uk&oq=nhs+si&gs_lcp=Cgdnd3Mtd2l6EAMYADIECCMQJzIECCMQJzIECCMQJzICCAAyAggAMgIIADICCAAyAggAMgIIADICCAA6BwgjELADECc6BwgAELADEEM6BwgAEEcQsAM6BQgAEJECOgsILhCxAxDHARCjAjoFCAAQsQM6CAgAELEDEIMBOg4ILhCxAxCDARDHARCjAjoICC4QsQMQgwE6CgguELEDEIMBEEM6BAgAEEM6BwgAELEDEEM6CggAELEDEIMBEEM6BQgAEMkDOgQIABAKOggIABAWEAoQHjoGCAAQFhAeUJ7XDljT6w5g6fEOaAVwAXgAgAHLAogBow2SAQcxLjUuMi4xmAEAoAEBqgEHZ3dzLXdpesgBCsABAQ&sclient=gws-wiz'
# req = requests.get(url)
# soup = BeautifulSoup(req.content, 'html.parser')
# # find the headlines
# # results = soup.find_all('div', attrs={"class":"home-summary-row"})

# import re

# for a in soup.find_all('a', href=True):
#     if "https://www.dailymail.co.uk/" in a['href']:
#         href = a['href']
#         real_href = re.split("//", href)
#         print(href)
#         print("---------------------------")
                        



In [30]:
import time 
from random import randint

i = 0
for website in websites:
    for x in range(26):
        search_result = search_selenium('nhs', website)
        start_date = search_days[i]
        end_date = search_days[i + 1]
        dated_website = enter_date(search_result, start_date, end_date)
        i += 1
        print(i)
        no_more_pages = False
        while no_more_pages == False:
            list_articles, driver = find_hrefs(dated_website, list_articles)
#             random_i = randint(5, 15)
            time.sleep(random_i)
            no_more_pages = next_page(driver)

print(i)
        

TimeoutException: Message: 


In [74]:
# url = 'https://www.dailymail.co.uk/news/article-9320239/NHS-hospitals-London-told-prepare-possible-surge-Covid-patients-later-year.html'

In [88]:
# import requests
# from bs4 import BeautifulSoup
# req = requests.get(daily_mail[701])
# soup = BeautifulSoup(req.content, 'html.parser')
# # find the headlines
# soup.find('h2').text

# # find the straplines
# for ultag in soup.find_all('ul', {'class': 'mol-bullets-with-font'}):
#     for litag in ultag.find_all('li'):
#         print(litag.text)
#         print('----------------------------')

# # get the timestamp
# date_time = soup.find('time').text
# date = date_time.split(',')[1]
# date


# # soup = BeautifulSoup(url2)
# # print(soup)
# # headline_selector = soup.find_all('div', class_='//*[@id="js-article-text"]')
# # print(headline_selector)

In [75]:
# driver = webdriver.Chrome(PATH)
# # will search on google
# driver.get('https://www.google.com/search?q=nhs+site:theguardian.com&tbs=cdr:1,cd_min:03-23-2019,cd_max:03-30-2019&ei=IgBAYKqiFrPPxgOSuoLYBQ&start=220&sa=N&ved=2ahUKEwiqu7GQiZXvAhWzp3EKHRKdAFs4jgIQ8tMDegQIBhBD&biw=1036&bih=529&dpr=1.5')
# element = driver.find_element_by_id('hdtb-tls')
# driver.execute_script("arguments[0].click();", element)
# #sleep
# random_i = randint(5, 15)
# driver.implicitly_wait(random_i)
# # Click on the date bar
# element2 = driver.find_element_by_class_name('hdtb-mn-hd')
# driver.execute_script("arguments[0].click();", element2)
# # Find the Customer Range menu
# element3 = driver.find_element_by_class_name('ErsxPb.gvybPb')
# driver.execute_script("arguments[0].click();", element3)
# #sleep
# random_i = randint(5, 15)
# driver.implicitly_wait(random_i)
# # Click the Customer Range menu
# try:
#     element4 = WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH, '//*[@id="lb"]/div[2]/g-menu/g-menu-item[7]/div/div/span')))
#     driver.execute_script("arguments[0].click();", element4)

# except:
#     element4 = WebDriverWait(driver, 25).until(EC.presence_of_element_located((By.XPATH, '//*[@id="ow54"]/span')))
#     driver.execute_script("arguments[0].click();", element4)
# element5 = driver.find_element_by_xpath('//*[@id="OouJcb"]')
# element5.send_keys(start_date)
# # Input End Date
# element6 = driver.find_element_by_xpath('//*[@id="rzG2be"]')
# element6.send_keys(end_date)
# # Random Sleep
# random_i = randint(5, 15)
# driver.implicitly_wait(random_i)
# # Submit Date
# element7 = driver.find_element_by_xpath('//*[@id="T3kYXe"]/g-button')
# driver = driver.execute_script("arguments[0].click();", element7)


# Practise Script

In [76]:
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from fake_useragent import UserAgent

# options = Options()
# ua = UserAgent()
# userAgent = ua.random
# print(userAgent)
# options.add_argument(f'user-agent={userAgent}')
# driver = webdriver.Chrome(chrome_options=options, executable_path=PATH)
# driver.get("https://www.google.co.in")
# driver.quit()


In [77]:
# from selenium import webdriver
# from selenium.webdriver.common.action_chains import ActionChains
# import time

# driver = webdriver.Chrome(PATH)

# driver.get('http://www.google.com')

# search = driver.find_element_by_name('q')
# # Enter search 
# search.send_keys('nhs site:theguardian.com')
# # hit return after you enter search text
# search.send_keys(Keys.RETURN) 
# #Sleep
# time.sleep(10)
# # Find the tool bar
# element = driver.find_element_by_id('hdtb-tls')
# driver.execute_script("arguments[0].click();", element)
# # Click on the date bar
# element2 = driver.find_element_by_class_name('hdtb-mn-hd')
# driver.execute_script("arguments[0].click();", element2)
# # Find the Custom Range menu
# element3 = driver.find_element_by_class_name('ErsxPb.gvybPb')
# driver.execute_script("arguments[0].click();", element3)
# # Click the Custom Range menu
# time.sleep(3)
# element4 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="lb"]/div[2]/g-menu/g-menu-item[7]/div/div/span')))
# # element4 = driver.find_element_by_xpath('//*[@id="lb"]/div[2]/g-menu/g-menu-item[7]/div/div/span')
# driver.execute_script("arguments[0].click();", element4)
# # Input Start Date
# element5 = driver.find_element_by_xpath('//*[@id="OouJcb"]')
# element5.send_keys('23-03-2019')
# # Input End Date
# element6 = driver.find_element_by_xpath('//*[@id="rzG2be"]')
# element6.send_keys('23-03-2019')
# time.sleep(5)

# # Submit Date
# element7 = driver.find_element_by_xpath('//*[@id="T3kYXe"]/g-button')
# driver.execute_script("arguments[0].click();", element7)


# ## get hrefs
# results = driver.find_elements_by_css_selector('div.g')
# time.sleep(5)
# for i in range(len(results)):
#     link = results[i].find_element_by_tag_name("a")
#     href = link.get_attribute("href")
#     print(href)
    
# # Select next page

# # while True:
# try:
#     next_page = driver.find_element_by_xpath("//*[@id='pnnext']").get_attribute("href")
#     driver.get(next_page)

# except:
#     print("no more pages left")
#     driver.quit()
#     False





In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time

driver = webdriver.Chrome(PATH)

driver.get('http://www.google.com')

search = driver.find_element_by_name('q')
# Enter search 
search.send_keys('nhs site:dailymail.co.uk')
# hit return after you enter search text
search.send_keys(Keys.RETURN) 
#Sleep
time.sleep(3)
# Find the tool bar
element = driver.find_element_by_id('hdtb-tls')
driver.execute_script("arguments[0].click();", element)
# Click on the date bar
element2 = driver.find_element_by_class_name('hdtb-mn-hd')
driver.execute_script("arguments[0].click();", element2)
# Find the Customer Range menu
element3 = driver.find_element_by_class_name('ErsxPb.gvybPb')
driver.execute_script("arguments[0].click();", element3)
# Click the Customer Range menu
time.sleep(3)
element4 = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="lb"]/div/g-menu/g-menu-item[7]/div/div/span')))
# element4 = driver.find_element_by_xpath('//*[@id="lb"]/div[2]/g-menu/g-menu-item[7]/div/div/span')
driver.execute_script("arguments[0].click();", element4)
# Input Start Date
element5 = driver.find_element_by_xpath('//*[@id="OouJcb"]')
element5.send_keys('23-03-2019')
# Input End Date
element6 = driver.find_element_by_xpath('//*[@id="rzG2be"]')
element6.send_keys('30-03-2019')
time.sleep(5)

# Submit Date
element7 = driver.find_element_by_xpath('//*[@id="T3kYXe"]/g-button')
driver.execute_script("arguments[0].click();", element7)


## get hrefs
results = driver.find_elements_by_css_selector('div.g')
time.sleep(5)
for i in range(len(results)):
    link = results[i].find_element_by_tag_name("a")
    href = link.get_attribute("href")
    print(href)

while True:
    try:
        next_page = driver.find_element_by_xpath("//*[@id='pnnext']").get_attribute("href")
        driver.get(next_page)
        results = driver.find_elements_by_css_selector('div.g')
        time.sleep(5)
        for i in range(len(results)):
            link = results[i].find_element_by_tag_name("a")
            href = link.get_attribute("href")
            print(href)

    except:
        print("no more pages left")
        driver.quit()


https://www.dailymail.co.uk/news/nhs/index.html
https://www.dailymail.co.uk/health/article-9339773/DR-ROB-GALLOWAY-vaccine-triumph-shows-NHS-run.html
https://www.dailymail.co.uk/tvshowbiz/article-9336561/Tom-Cruise-surprises-NHS-staff-secret-visit-thank-working-pandemic.html
https://www.dailymail.co.uk/news/article-9338717/Boris-Johnson-refuses-rule-Covid-bonus-NHS-staff-amid-fury-1-cent-pay-offer.html
https://www.dailymail.co.uk/news/article-9043583/NHS-data-suggests-hospitals-England-December.html
https://www.google.com/search?q=COVID-19&sa=X&ved=2ahUKEwiorfKE46jvAhWKlosKHa5GBlYQzmcoAHoECBgQAQ
https://www.gov.uk/government/news/nhs-covid-19-app-launches-across-england-and-wales#:~:text=As%20well%20as%20contact%20tracing,be%20provided%20to%20relevant%20advice
https://www.gov.uk/guidance/nhs-test-and-trace-how-it-works#:~:text=We%20will%20contact%20you%20by,to%20continue%20the%20call.
https://www.gov.uk/government/news/new-lighthouse-labs-to-boost-nhs-test-and-trace-capacity#:~:text=%E

# Get headlines from links

In [ ]:
#regex for partial match with class_name

In [79]:
# import re

In [80]:
# real_headlines = []

In [81]:
# driver = webdriver.Chrome(PATH)
# driver.maximize_window()

# wait = WebDriverWait(driver, 3)
# presence = EC.presence_of_element_located
# visible = EC.visibility_of_element_located

# for i in list_headlines:
#     driver.get(i)
#     WebDriverWait(driver, 10)
#     print(i)
    
#     try:
#         divs = driver.find_element_by_css_selector("[class*='e-headline']")
#         real_headlines.append(divs.text)
# #         print(divs.text)
        
#     except:
#         divs = None


In [82]:
# real_headlines
# myset = set(real_headlines)
# len(myset)

In [84]:
# len(real_headlines)

In [85]:
# driver = webdriver.Chrome(PATH)
# driver.maximize_window()

# wait = WebDriverWait(driver, 5)
# presence = EC.presence_of_element_located
# visible = EC.visibility_of_element_located

# for website in websites_google_search:
#     driver = webdriver.Chrome(PATH)
#     driver.maximize_window()
#     for day in search_days:
#         driver.get(f'https://www.google.com/search?q=nhs+site:{website}&tbs=cdr:1,cd_min:{day},cd_max:{day}&sxsrf=ALeKk02Trw5YASvAO8g4qyRyb2eRf-Q2cg:1614704439315&ei=N28-YKrsEtHxxgPilIboDA&start=90&sa=N&ved=2ahUKEwjq45bkipLvAhXRuHEKHWKKAc0Q8tMDegQIAxBF&biw=1280&bih=614')
#         while True:
#             random_i = randint(5, 15)
#             WebDriverWait(driver, random_i)
#             next_page_btn =driver.find_elements_by_xpath("//a[@id='pnnext']")
#             if 
            
#             elif len(next_page_btn) <1:
#                 print("no more pages left")
#                 driver.quit()
#                 break

#             else:
#                 random_i = randint(5, 15)
#                 WebDriverWait(driver, random_i)
#                 results = driver.find_elements_by_css_selector('div.g')
#                 for i in range(len(results)):
#                     link = results[i].find_element_by_tag_name("a")
#                     href = link.get_attribute("href")
#                     if website in href:
#                         print(href)
#                         list_headlines.append(href)
#                 next_page = driver.find_element_by_xpath("//*[@id='pnnext']").get_attribute("href")
#                 driver.get(next_page)
            
#             href = link.get_attribute("href").click()
#             random_i = randint(3, 5)
#             WebDriverWait(driver, random_i)
#             new_ = next_page.get_attribute("href").click()
#             print(new_)
#             driver.Click()
#             driver.next_page[0].get_attribute("href").click()
        
# with open("all_h_refs.txt", "w") as output:
#     output.write(str(list_headlines))



In [86]:

# for website in websites_google_search:
#     driver = webdriver.Chrome(PATH)
#     driver.maximize_window()
#     for day in search_days:
#         driver.get(f'https://www.google.com/search?q=nhs+site:{website}&tbs=cdr:1,cd_min:{day},cd_max:{day}&sxsrf=ALeKk02Trw5YASvAO8g4qyRyb2eRf-Q2cg:1614704439315&ei=N28-YKrsEtHxxgPilIboDA&start=90&sa=N&ved=2ahUKEwjq45bkipLvAhXRuHEKHWKKAc0Q8tMDegQIAxBF&biw=1280&bih=614')
#         while True:
#             random_i = randint(5, 15)
#             WebDriverWait(driver, random_i)
#             one_page_link = driver.find_elements_by_class_name('LC20lb')
# #             if len(one_page_link) > 0: 
# #                 random_i = randint(5, 15)
# #                 WebDriverWait(driver, random_i)
# #                 results = driver.find_elements_by_css_selector('div.g')
# #                 for i in range(len(results)):
# #                     link = results[i].find_element_by_tag_name("a")
# #                     href = link.get_attribute("href")
# #                     if website in href:
# #                         print(href)
# #                         list_headlines.append(href)
# #                 next_page = driver.find_element_by_xpath("//*[@id='pnnext']").get_attribute("href")
# #                 driver.get(next_page)

# #             else:
# #                 print("no more pages left")
# #                 driver.quit()
# #                 break
            
# with open("all_h_refs.txt", "w") as output:
#     output.write(str(list_headlines))



In [87]:
# driver = webdriver.Chrome(PATH)
# driver.maximize_window()

# wait = WebDriverWait(driver, 5)
# presence = EC.presence_of_element_located
# visible = EC.visibility_of_element_located

# # Navigate to url with video being appended to search_query
# for website in websites_google_search:
#     wait

#     page_number_found = True
#     while page_number_found == True:
#         for date in search_days:
#             try:
#                 random_i = randint(5, 15)
#                 WebDriverWait(driver, random_i)
#                 driver.get(f'https://www.google.com/search?q=nhs+site:{website}&tbs=cdr:1,cd_min:{date},cd_max:{date}&sxsrf=ALeKk02IWAdbzTcOOTXprjips1JZGT3XxA:1614530791185&ei=58g7YMLpCq2O1fAP8JuQ0Ak&start={google_page_number}&sa=N&ved=2ahUKEwiC-aXyg43vAhUtRxUIHfANBJo4ChDy0wN6BAgDEDc&biw=1280&bih=614')
#                 results = driver.find_elements_by_css_selector('div.g')
#                 for i in range(len(results)):
#                     wait
#                     link = results[i].find_element_by_tag_name("a")
#                     href = link.get_attribute("href")
#                     print(href)


#             except:
#                 page_number_found == False
#                 break

#             google_page_number += 10